<a href="https://colab.research.google.com/github/JOE1904/JOE1904/blob/main/Final_Code_Architecture(Fair).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
!pip install requests

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install wikipedia-api

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import pandas as pd
import re

## **STEP: 1A  Converting PDF To Text or Taking in Text Files:**

In [ ]:
import PyPDF2

# Open the PDF file
pdf_file = open('example.pdf', 'rb')

# Create a PDF reader object
pdf_reader = PyPDF2.PdfFileReader(pdf_file)

# Initialize an empty string to store the text
text = ""

# Loop through each page and extract text
for page_number in range(pdf_reader.getNumPages()):
    page = pdf_reader.getPage(page_number)
    text += page.extractText()

# Close the PDF file
pdf_file.close()


## **STEP : 1B Converting PDF To Text or Operating Directly on Text Files:**

In [ ]:
# Converting PDF To Text File:
with open('output.txt', 'w') as file:
    # Write your text to the file
    file.write(text)

# The file is automatically closed when the "with" block is exited
print("Text has been written to 'output.txt'.")

# Working on the Text File:
file_path =  "/content/sample_data/data_sci.txt" # Replace '/content/sample_data/data_sci.txt' with the path to your text file
with open(file_path, 'r') as file:
    # Read the entire file content into a string
    file_content = file.read()


## **STEP: 1C Text-Preprocessing (The Necessary Things)**

**The following steps will be carried out in this block of code:** <br>
  - The text will iterate through a loop where all the tokens excluding numerical values will be taken out.
  - The words/tokens taken out will undergo the Name-recognition Step.
  - The text/corpus will remain the same. This step is done for the Highlight step in the website.

In [ ]:
# Remove numerical values, punctuation, and special characters:
cleaned_text = re.sub(r'[^a-zA-Z\s]', '', re.sub(r'\d+', '', file_content))

# Remove all numerical values from the text:
text_without_numbers = re.sub(r'\d+', '', cleaned_text)

## **STEP: 2 Name- Entity Recognition:**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

# Putting corpus into pipeline for entity recognition:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Creating A DatFrame for the Name- Entities Created :
data = [(result['entity'], result['word']) for result in ner_results]

# Create a DataFrame
df = pd.DataFrame(data, columns=['Entity Type', 'Word'])
df

## **STEP:3 Generating/ Web-Scraping of each word from Wikipedia or Relevant Data Source :**

In [ ]:
                                                      # OPTION 1: Web-Scraping from Wikipedia

import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page you want to scrape
url = "https://en.wikipedia.org/wiki/Web_scraping"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the element containing the introduction paragraph
intro_paragraph = soup.find("p")

# Extract the text from the paragraph element
if intro_paragraph:
    intro_text = intro_paragraph.get_text()
    print(intro_text)
else:
    print("Introduction paragraph not found on the page.")


                                                      # OPTION - 2 From Wikipedia Library in Python:

import wikipediaapi

# Specify the language for Wikipedia (e.g., 'en' for English)
wiki_lang = wikipediaapi.Wikipedia('en')

# Specify the keyword you want to search for:

for keyword in df["tokens"]:
  search_keyword = keyword

  # Get the page object for the search keyword
  page = wiki_lang.page(search_keyword)

  # Check if the page exists and has an introduction section
  if page.exists():
    introduction_section = page.sections[0]  # The first section is usually the introduction

    # Extract the text of the introduction paragraph
    intro_text = introduction_section.text

    df["Intro_text"] = intro_text
  else:
    print("Page not found on Wikipedia for the given keyword.")

## **STEP: 4 Text Summarizer for the corpus for each Word:**

### Model 1: T5F Large

In [ ]:
# Load pre-trained T5 model and tokenizer:
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

for intro_text in df["Intro_text"]:
  inputs = tokenizer.encode("summarize: " + text_to_summarize, return_tensors="pt", max_length=1024, truncation=True)
  summary_ids = model.generate(inputs, max_length=120, min_length=150, length_penalty=2.0, num_beams=4, early_stopping=True)

  # Decode and print the summary
  summary_tf_small = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  df["Summary_tf-small"] = summary

### Model 2: BERT or RoBERTa

### Optimal Model: GPT-2 Summarizer: